# Using Hermione ML to treat missing values

![title](tenor.gif)

**It is necessary to change the working directory so the project structure works properly:**

In [1]:
import sys
sys.path.append("../../")

From this point, it's on you!

---

In [6]:
!pip install --upgrade category_encoders

  Using cached category_encoders-2.2.2-py2.py3-none-any.whl (80 kB)


You should consider upgrading via the 'C:\Users\marin.DESKTOP-JCC1RH5\anaconda3\python.exe -m pip install --upgrade pip' command.


In [10]:
!pip install mlflow

Processing c:\users\marin.desktop-jcc1rh5\appdata\local\pip\cache\wheels\69\38\7a\072b5863ca334d012821a287fd1d066cea33abdcda3ef2f878\querystring_parser-1.2.4-py3-none-any.whl
  Using cached waitress-1.4.4-py2.py3-none-any.whl (58 kB)
  Using cached gorilla-0.3.0-py2.py3-none-any.whl (11 kB)
  Using cached websocket_client-0.57.0-py2.py3-none-any.whl (200 kB)
  Using cached gitdb-4.0.5-py3-none-any.whl (63 kB)
  Using cached tabulate-0.8.7-py3-none-any.whl (24 kB)
  Using cached Mako-1.1.3-py2.py3-none-any.whl (75 kB)
  Using cached python_editor-1.0.4-py3-none-any.whl (4.9 kB)
  Using cached smmap-3.0.4-py2.py3-none-any.whl (25 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached isodate-0.6.0-py2.py3-none-any.whl (45 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)
  Created wheel for prometheus-flask-exporter: filename=prometheus_flask_exporter-0.18.1-py3-none-any.whl size=17163 sha256=838c79dfd2264ce1f2225a3400aa3a89ca274acd0d568dafb9

You should consider upgrading via the 'C:\Users\marin.DESKTOP-JCC1RH5\anaconda3\python.exe -m pip install --upgrade pip' command.


In [11]:
import pandas as pd

from ml.data_source.spreadsheet import Spreadsheet
from ml.preprocessing.preprocessing import Preprocessing
from ml.model.trainer import TrainerSklearn

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [30]:
df = Spreadsheet().get_data('../../../data/raw/train.csv')

In [13]:
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age'], dtype='object')

In [14]:
p = Preprocessing()

In [15]:
df = p.clean_data(df)
df = p.categ_encoding(df)

Cleaning data
Category encoding


In [16]:
df.head()

,Survived,Age,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male
0,0,22.0,0,0,1,0,1
1,1,38.0,1,0,0,1,0
2,1,26.0,0,0,1,1,0
3,1,35.0,1,0,0,1,0
4,0,35.0,0,0,1,0,1


In [17]:
X = df.drop(columns=["Survived"])
y = df["Survived"]

In [18]:
# Ensure the same random state passed to TrainerSkleran().train()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((499, 6), (215, 6), (499,), (215,))

In [19]:
rf = TrainerSklearn().train(X, y, classification=True, 
                            algorithm=RandomForestClassifier, 
                            preprocessing=p,
                           data_split=('train_test', {'test_size':.3}),
                           random_state=123)

C:\Users\marin.DESKTOP-JCC1RH5\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [20]:
rf.get_metrics()

{'accuracy': 0.7534883720930232,
 'f1': 0.6826347305389222,
 'precision': 0.7037037037037037,
 'recall': 0.6627906976744186,
 'roc_auc': 0.7993510005408329}

In [21]:
rf.get_columns()

['Age', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male']

In [22]:
rf.predict_proba(X_test, binary=True)

array([1.        , 0.        , 0.41595238, 1.        , 0.1       ,
       0.        , 0.        , 0.        , 0.        , 0.9       ,
       0.59047619, 0.11472222, 0.11472222, 1.        , 1.        ,
       0.32      , 0.        , 0.39411255, 1.        , 0.        ,
       0.195     , 1.        , 0.45833333, 0.9       , 1.        ,
       1.        , 0.        , 1.        , 0.        , 0.27248779,
       0.        , 0.        , 1.        , 0.05      , 0.        ,
       0.3       , 0.39411255, 0.        , 0.32477633, 0.        ,
       0.        , 0.        , 0.43333333, 0.        , 1.        ,
       1.        , 0.        , 1.        , 1.        , 0.36666667,
       0.        , 0.2       , 0.        , 0.1       , 1.        ,
       0.39411255, 1.        , 0.        , 0.11472222, 0.        ,
       1.        , 1.        , 0.        , 0.        , 0.8       ,
       0.        , 0.        , 1.        , 0.        , 0.39411255,
       0.        , 0.1       , 0.        , 0.35166667, 0.9    

In [23]:
# Predicting new data
def predict_new(X, model, probs=True):
    X = p.clean_data(X)
    X = p.categ_encoding(X)
    
    columns = model.get_columns()
    for col in columns:
        if col not in X.columns:
            X[col] = 0
    print(X)
    if probs:
        return model.predict_proba(X)
    else:
        return model.predict(X)

In [24]:
new_data = pd.DataFrame({
    'Pclass':3,
    'Sex': 'male',
    'Age':4
}, index=[0])

new_data

,Pclass,Sex,Age
0,3,male,4


In [25]:
predict_new(new_data, rf)

Cleaning data
Category encoding
   Age  Pclass_3  Sex_male  Pclass_1  Pclass_2  Sex_female
0    4         1         1         0         0           0


array([[0.64833333, 0.35166667]])